In [ ]:
# Title: Cellpose Image Processing Pipeline for Cytoplasm Mask Generation

# Description:
# This Python script processes cytoplasm images using a custom-trained Cellpose model to generate cell masks. 
# It allows the user to specify various parameters such as input/output directories, cell diameter, flow threshold, 
# cell probability threshold, and output format. The script leverages a custom model path and processes all single-channel 
# AF488 cytoplasm images (`*_c1_AF488.tif`) from the specified input directory. The generated masks are saved in the 
# chosen format (`png` or `tiff`) in the output directory.

# Model Training Workflow:
# - The Cellpose model was first trained on the base model `cyto2_cp3` (or Cyto 2).
# - Sample images were organized into a folder named `CellPose Training` for training purposes.
# - Three representative images (tiles) were selected to ensure good representation of cytoplasmic features.
# - These sample images were used to generate initial masks. The labels were manually corrected to improve accuracy.
# - After correcting the labels, the `Train new model with image + masks in folder` option was used in the Cellpose GUI.
# - This process generated the custom-trained model, which is now utilized in this pipeline.

# Key Features:
# - **Custom Model Loading**: Uses the custom-trained Cellpose model for cytoplasm mask generation.
# - **Parameter Flexibility**: Allows enforcement of fixed cell diameters, adjusting thresholds, and selecting output formats.
# - **Batch Processing**: Processes all matching images in the input directory with real-time progress updates using `tqdm`.
# - **Output Customization**: Masks are saved as 16-bit images to accommodate labels beyond 255.

In [5]:
from tqdm import tqdm
import time
import os
import glob
from skimage.io import imread, imsave
from cellpose import models
import numpy as np

def process_images(
    input_dir,
    output_dir,
    custom_model_path,
    channels=[0, 0],
    diameter=200,     #Def target diameter
    enforce_diameter=True,
    flow_threshold=0.4,
    cellprob_threshold=0.0,
    output_format="png"
):
    """
    Process images in input_dir using a custom Cellpose model, saving the masks
    into output_dir, optionally enforcing a fixed diameter and choosing output format.
    
    Parameters
    ----------
    input_dir : str
        Folder containing input images (e.g., `*_c1_AF488.tif`).
    output_dir : str
        Folder where masks will be saved.
    custom_model_path : str
        Path to your custom-trained Cellpose model (a .pth file).
    channels : list of int
        Channel specification for Cellpose; for single-channel use [0,0].
    diameter : float
        Desired cell diameter in pixels. Used only if enforce_diameter=True.
    enforce_diameter : bool
        If True, will use the provided 'diameter' value rather than letting Cellpose guess.
    flow_threshold : float
        Flow threshold passed to Cellpose `eval()`.
    cellprob_threshold : float
        Cell probability threshold passed to Cellpose `eval()`.
    output_format : str
        File format for output masks. Either 'png' or 'tiff'.
    """

    # Validate output format
    if output_format.lower() not in ["png", "tiff", "tif"]:
        raise ValueError(f"output_format must be 'png' or 'tiff', got {output_format}")

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Load the custom model directly as CellposeModel (bypassing _size.npy check)
    model = models.CellposeModel(
        gpu=True, 
        pretrained_model=custom_model_path
    )

    # Decide on diameter
    if enforce_diameter:
        # Always use this diameter
        final_diameter = diameter
    else:
        # Let the model pick the diameter if diameter=0
        if diameter == 0:
            final_diameter = model.diam_labels
        else:
            final_diameter = diameter

    # Gather input files
    image_files = glob.glob(os.path.join(input_dir, "*_c1_AF488.tif"))
    print(f"Found {len(image_files)} files to process.")

    # Start a timer for overall processing
    start_time = time.time()

    #for img_path in image_files:
    for img_path in tqdm(image_files, desc="Processing images", unit="image"):
        # Read image
        img = imread(img_path)

        # Run Cellpose
        masks, flows, styles = model.eval(
            img,
            diameter=final_diameter,
            channels=channels,
            flow_threshold=flow_threshold,
            cellprob_threshold=cellprob_threshold
        )

        # Prepare output filename
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        new_base_name = base_name.replace("Phenotype_Tile-", "Phenotype_CellMask_Tile-")
        
        if output_format.lower() in ["tiff", "tif"]:
            save_path = os.path.join(output_dir, new_base_name + ".tiff")
        else:
            save_path = os.path.join(output_dir, new_base_name + ".png")
        
        # Save mask (16-bit to be safe if labels go above 255)
        imsave(save_path, masks.astype(np.uint16))

        #print(f"Saved mask to: {save_path}")
    # End timer
    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nProcessing completed in {total_time:.2f} seconds.")

if __name__ == "__main__":
    # --------------------------
    # USER-SPECIFIC SETTINGS
    # --------------------------
    input_dir = r"D:/Easin/OPS_Nov_2024/TIFF/gamma_adjusted_images/cells"
    output_dir = r"D:/Easin/OPS_Nov_2024/TIFF/gamma_adjusted_images/cells/masks2"
    custom_model_path = r"S:/CellPose Training Images/CellPoseTraining2/TIFF/gamma_adjusted_images/Cytoplasm/models/CP_20250107_123623"
    os.makedirs(output_dir, exist_ok=True)
    
    # Choose your desired parameters:
    channels = [0, 0]               # Single-channel cytoplasm images
    diameter = 200                  # enforce 200 px diameter
    enforce_diameter = True         # True => override model's guess
    flow_threshold = 0.6
    cellprob_threshold = 0.0
    output_format = "png"          # Options: 'png' or 'tiff'

    # Run the pipeline
    process_images(
        input_dir=input_dir,
        output_dir=output_dir,
        custom_model_path=custom_model_path,
        channels=channels,
        diameter=diameter,
        enforce_diameter=enforce_diameter,
        flow_threshold=flow_threshold,
        cellprob_threshold=cellprob_threshold,
        output_format=output_format
    )


Found 360 files to process.


Processing images:   0%|          | 0/360 [00:00<?, ?image/s]C:\Users\syede\AppData\Local\Temp\ipykernel_10756\229957258.py:96: UserWarning: D:/Easin/OPS_Nov_2024/TIFF/gamma_adjusted_images/cells/masks2\Phenotype_CellMask_Tile-023_c1_AF488.png is a low contrast image
  imsave(save_path, masks.astype(np.uint16))
Processing images:   0%|          | 1/360 [00:23<2:22:44, 23.86s/image]C:\Users\syede\AppData\Local\Temp\ipykernel_10756\229957258.py:96: UserWarning: D:/Easin/OPS_Nov_2024/TIFF/gamma_adjusted_images/cells/masks2\Phenotype_CellMask_Tile-024_c1_AF488.png is a low contrast image
  imsave(save_path, masks.astype(np.uint16))
Processing images:   1%|          | 2/360 [01:05<3:24:34, 34.29s/image]C:\Users\syede\AppData\Local\Temp\ipykernel_10756\229957258.py:96: UserWarning: D:/Easin/OPS_Nov_2024/TIFF/gamma_adjusted_images/cells/masks2\Phenotype_CellMask_Tile-025_c1_AF488.png is a low contrast image
  imsave(save_path, masks.astype(np.uint16))
Processing images:   1%|          | 3/360


Processing completed in 51953.52 seconds.
